### Installing all the libraries

In [ ]:
!pip install datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
!pip install langchain
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os
import re

import pandas as pd

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

from langchain.docstore.document import Document

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import time

import os
# from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL2.git

Cloning into 'DeepSeek-VL2'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 165 (delta 22), reused 13 (delta 13), pack-reused 138 (from 1)
Receiving objects: 100% (165/165), 31.04 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
import os

# Define the directory path
dir_path = "/content/DeepSeek-VL2"

# Check if it exists, if not, create it
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# Change to the directory
%cd {dir_path}


/content/DeepSeek-VL2


In [ ]:
pip install -e .

Obtaining file:///content/DeepSeek-VL2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5

In [ ]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl.utils.io import load_pil_images


# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-small"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

## single image conversation example
conversation = [
    {
        "role": "<|User|>",
        "content": "<image>\n<|ref|>The giraffe at the back.<|/ref|>.",
        "images": ["./images/visual_grounding.jpeg"],
    },
    {"role": "<|Assistant|>", "content": ""},
]

## multiple images (or in-context learning) conversation example
# conversation = [
#     {
#         "role": "User",
#         "content": "<image_placeholder>A dog wearing nothing in the foreground, "
#                    "<image_placeholder>a dog wearing a santa hat, "
#                    "<image_placeholder>a dog wearing a wizard outfit, and "
#                    "<image_placeholder>what's the dog wearing?",
#         "images": [
#             "images/dog_a.png",
#             "images/dog_b.png",
#             "images/dog_c.png",
#             "images/dog_d.png",
#         ],
#     },
#     {"role": "Assistant", "content": ""}
# ]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


ModuleNotFoundError: No module named 'deepseek_vl'

In [ ]:
# from google.colab import userdata


# To use it make sure to add your Gemini api key in the colab secrets with `GOOGLE_API_KEY` as the variable name

# Load the language model
# model = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash-exp", temperature=0.5
# )

model = genai.GenerativeModel(model_name="gemini-1.5-pro")
# model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

genai.configure(api_key=google_api_key)


In [ ]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/Rajarshi-Roy-research/Som_bench_refcocog_refseg/data/train-00000-of-00001.parquet")

In [ ]:
df.head()

,id,image,obj_text,ref_ids,json_data
0,000000007601,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[A black cow that is only half visible standin...,"[3, 1, 2]","[{'ref_id': 3, 'text': ['A black cow that is o..."
1,000000031187,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[little girl in light green shirt; Little Girl...,"[2, 1, 3, 4]","[{'ref_id': 2, 'text': ['little girl in light ..."
2,000000066669,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[an orange cat stuffed animal; The teddy bear ...,"[4, 5]","[{'ref_id': 4, 'text': ['an orange cat stuffed..."
3,000000017997,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[The giraffe standing in the middle.; A giraff...,"[3, 2]","[{'ref_id': 3, 'text': ['The giraffe standing ..."
4,000000032707,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[skier in orange coat in the right hand pictur...,"[2, 1]","[{'ref_id': 2, 'text': ['skier in orange coat ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         100 non-null    object
 1   image      100 non-null    object
 2   obj_text   100 non-null    object
 3   ref_ids    100 non-null    object
 4   json_data  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [ ]:
import re
import ast

def extract_list_from_string(text):
    """
    Extracts a list from a string by finding the first occurrence of square brackets.

    Parameters:
        text (str): The input string containing a Python list.

    Returns:
        list: Extracted list if found, otherwise an empty list.
    """
    match = re.search(r"\[.*?\]", text, re.DOTALL)
    if match:
        try:
            return ast.literal_eval(match.group(0))  # Safely parse the list
        except (SyntaxError, ValueError):
            return []
    return []

# Example usage
text = "```python\n[3, 1, 2]\n```"
print(extract_list_from_string(text))  # Output: [3, 1, 2]
print(type(extract_list_from_string(text)))  # Output: [3, 1, 2]


[3, 1, 2]
<class 'list'>


In [ ]:
import httpx
import base64




def get_prompt(obj_list):
  prompt=f"""I have labeled a bright numeric ID at the center for each visual object in the image. \nPlease tell me the IDs for:"""

  for i in obj_list:
    prompt += f"\n-{i}\n"

  prompt+= "\nPlease, properly give answer in the form of just a python list format accoriding to the order of objects asked\nAnswer Example:`[1,2,4..]`"

  # print(prompt)
  return prompt

def get_response(prompt,img_bytes):

  # Extract image data and encode to base64
  images_data = [
      {
          'mime_type': 'image/jpeg',
          'data': base64.b64encode(img_bytes).decode("utf-8")
      }
  ]

  # Generate response
  response = model.generate_content(images_data + [prompt])
  text = response.candidates[0].content.parts[0].text
  # print(text)

  return extract_list_from_string(text)

def get_prediction(index):

  img_bytes = df.loc[index,"image"]["bytes"]
  obj_list = df.loc[index,"obj_text"]

  prompt = get_prompt(obj_list)

  return get_response(prompt,img_bytes)


In [ ]:
get_prediction(1)

KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         100 non-null    object
 1   image      100 non-null    object
 2   obj_text   100 non-null    object
 3   ref_ids    100 non-null    object
 4   json_data  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [ ]:
from tqdm import tqdm
import time
error_indexes = []
error_titles = []

progress_bar = tqdm(total=len(df), desc="Processing")

# Wrap df.iterrows() with tqdm to show progress
for index, row in df.iterrows():
    # if pd.notna(row['image']):  # Check if 'image' is not NaN

    if pd.notna(row['image']) and pd.isna(row['gemini_ref_ids_pred']):  # Check if 'image' is not NaN

        if index % 2 == 0:  # Update tqdm every 100 iterations
          time.sleep(2)

        try:
          response = get_prediction(index)

          df.at[index, 'gemini_ref_ids_pred'] = f"{response}"

        except Exception as e:
          print(f"Error processing row {index}: {e}")
          error_indexes.append(index)
          error_titles.append(row['obj_text'])
          break

    progress_bar.update(1)


progress_bar.close()

Processing:  13%|█▎        | 13/100 [02:14<14:57, 10.32s/it]


KeyboardInterrupt: 

In [ ]:
get_prediction(20)

KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   100 non-null    object
 1   image                100 non-null    object
 2   obj_text             100 non-null    object
 3   ref_ids              100 non-null    object
 4   json_data            100 non-null    object
 5   gemini_ref_ids_pred  17 non-null     object
dtypes: object(6)
memory usage: 4.8+ KB


In [ ]:
response

[45, 89, 45, 45, 45, 45, 15, 78]

In [ ]:
response

[3, 1, 2]

In [ ]:
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 14 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   section                                                        932 non-null    object
 1   title                                                          932 non-null    object
 2   abstract                                                       932 non-null    object
 3   url                                                            932 non-null    object
 4   Human story fetched                                            932 non-null    object
 5   AI generated article with context(Human Story)                 912 non-null    object
 6   AI generated story(without context)                            912 non-null    object
 7   Web retrieved URLs                                             932 non-